# Parameters

Description of the parameters:

**GridParameters:**

| Parameter      | Description |
| :------------- | :---------- |
| dim            | Dimension of the grid. Supported dimensions: 1D (dim=1) and 2D (dim=2). |
| L              | The length of the grid in grid cells in each dimension. |
| sites          | An array containing the positions of the binding sites. The positions can be given using linear (1D) indices or indices with the same dimension (nD) as the grid. In the case of 1D indices, sites will be a 1D array with length equal to the number of binding sites. In the case of nD indices, sites will be a 2D array where the length along the first dimension is equal to the number of binding sites and the length along the second dimension is equal to the dimension of the grid. |
| IC_mode        | Mode for generating initial conditions. Options: 'fill fraction', 'probability', 'grid'. When the mode is 'fill fraction', the grid is filled randomly to obtain the given fill fraction (fraction of sites filled). When the mode is 'probability', a grid is randoly chosen from a list of grids according to the given probabilities. When the mode is 'grid', the given grid is used as initial condition. |
| periodic       | Boolean that indicates whether periodic boundary conditions should be used (True) or not (False) (default = False). |
| do_htf_update  | Boolean that indicates whether the $h_{tf}$ value should be updated when molecules appear on or disappear from the grid. (default = False). True = Finite bath, False = Infinite bath. |
| beta           | Multiplication factor to convert energies to units of $k_BT$ (default = 1). |
| grid_cell_size | The size of a single grid cell in $\mu$m (default = 0.004). |
| R              | The size of the nucleus in $\mu$m (default = 3). |

**SimulationParameters:**

| Parameter     | Description |
| :------------ | :---------- |
| runs          | Number of runs of the simulation. |
| T             | Runtime in time units. |
| T_trans       | Time of transient in time units, which should be removed when calculating model results (default = None). |
| max_timesteps | Maximum number of timesteps for which to run the simulation (default = int(1e6)). |

**VariedParameters:**

| Parameter     | Description |
| :------------ | :---------- |
| htf_list     | List of the different htf values used for the simulations. For each htf in htf_list, the simulations will be repeated for all sets of eb and J. |
| eb_list      | List of the different binding energies used for the simulations. |
| J_list       | List of the different clustering strengths used for the simulations. J_list must have the same length as eb_list, where values with the same index form a set. |
| IC_prms_list | Dictionary containing the different IC_prms used for the different simulations.  IC_prms_list contains IC_prms for every combination of htf, eb and J. The keys are given by f'J{J}eb{eb}htf{htf}'. |

### Stochastic simulations

Parameter values used for the stochastic simulations:

| ID   | Binding sites (N) | Padding     | Periodic    | Bath (do_htf_update) | Expression (N_exp) | Initial Conditions |
| :--- | :---------------: | :---------: | :---------- | :------------------- | :----------------- | :----------------- |
| SSNN | 6                 | 0           | False       | Infinite (False)     | Occupancy (None)   | SteadyState        |
| SS2N | 6                 | 2           | False       | Infinite (False)     | Occupancy (None)   | SteadyState        |
|      |                   |             |             |                      |                    |                    |
| N6NN | 6                 | 0           | False       | Infinite (False)     | All-occupied (6)   | SteadyState        |
| N62N | 6                 | 2           | False       | Infinite (False)     | All-occupied (6)   | SteadyState        |

Insert/Modify these parameters in the script below, to create the required metadata files.

# Scripts

In [1]:
import math as m
import numpy as np
import scipy as sp
from datetime import date
import pickle

import os
import sys
sys.path.append('../Parameters/')
sys.path.append('../Parameters/Bcd_data/')
sys.path.append('../AnalyticalModel/')

import ParametersFromLiterature as prms
from Parameters import GridParameters, SimulationParameters, VariedParameters
from Metadata import write_metadata
from ShiftCompensation import ShiftCompensation
from bcd import read_bcd_data, discretize_concentrations

from AnalyticalSimulationRun import get_steady_states

In [2]:
# Calculate the htf values for the simulation based on the discretization of the bicoid data. 
# For Bcd mean and variance, we use data kindly provided by Thomas Gregor, from the publication:
# Gregor T, Tank DW, Wieschaus EF, Bialek W. Probing the limits to positional information. 
# Cell. 2007 Jul 13;130(1):153-64. doi: 10.1016/j.cell.2007.05.025. PMID: 17632062; PMCID: PMC2253670.

# The variable number_of_samples gives the number of samples between the relative concentrations 0 and 1.
# A relative concentration of 1 is assumed to represent max_num_of_molecules molecules.
# Extra bins are added for values above 1, to make sure the spread around a mean close to 1 is properly sampled.
# The total number of bins/samples is thus higher than given by the varible number_of_samples.

def get_htfs(max_num_of_molecules, number_of_samples, beta, N_sites_tot):
    bcd_data = read_bcd_data('../../Parameters/Bcd_data/TG_normBcd')
    rel_conc_bins, threshold_pos, rel_threshold_conc = discretize_concentrations(bcd_data, number_of_samples)
    
    # We assume that 1 in the data corresponds to max_num_of_molecules.
    rel_concentrations = rel_conc_bins[:-1]+1/(2*number_of_samples)
    molecules          = rel_concentrations*max_num_of_molecules
    concentrations     = molecules/N_sites_tot
    bcd_htfs           = prms.c_to_htf(beta, concentrations)
    treshold_conc      = rel_threshold_conc*max_num_of_molecules/N_sites_tot
    bcd_htf_05         = prms.c_to_htf(beta, treshold_conc)
    
    concentration_conversion = 1/(prms.V_site*prms.N_Avogadro/(10**24))
    print('Relative concentrations:\n', molecules/max_num_of_molecules, '\n')
    print('Threshold position (x/L):', threshold_pos)
    print('Threshold concentration (molecules/site):', treshold_conc)
    print(f'Threshold concentration (nM, R={prms.R} um):', treshold_conc*concentration_conversion)
    print(f'Max mean concentration (nM, {max_num_of_molecules} molecules per nucleus):', (max_num_of_molecules/N_sites_tot)*concentration_conversion)
    print('Max concentration (nM):', (rel_conc_bins[-1]*max_num_of_molecules/N_sites_tot)*concentration_conversion)
    print(f'Total number of bins ({number_of_samples} bins between 0 and 1):', len(rel_conc_bins)-1, '\n')
    return bcd_htfs, bcd_htf_05

In [4]:
bcd_htfs, bcd_htf_05 = get_htfs(10000, 60, prms.beta, prms.N_sites_tot)
print('htf values:\n', bcd_htfs, '\n')
print('htf_05:', bcd_htf_05)

Relative concentrations:
 [0.00833333 0.025      0.04166667 0.05833333 0.075      0.09166667
 0.10833333 0.125      0.14166667 0.15833333 0.175      0.19166667
 0.20833333 0.225      0.24166667 0.25833333 0.275      0.29166667
 0.30833333 0.325      0.34166667 0.35833333 0.375      0.39166667
 0.40833333 0.425      0.44166667 0.45833333 0.475      0.49166667
 0.50833333 0.525      0.54166667 0.55833333 0.575      0.59166667
 0.60833333 0.625      0.64166667 0.65833333 0.675      0.69166667
 0.70833333 0.725      0.74166667 0.75833333 0.775      0.79166667
 0.80833333 0.825      0.84166667 0.85833333 0.875      0.89166667
 0.90833333 0.925      0.94166667 0.95833333 0.975      0.99166667
 1.00833333 1.025      1.04166667 1.05833333 1.075     ] 

Threshold position (x/L): 0.4824120603015075
Threshold concentration (molecules/site): 7.204998794474998e-07
Threshold concentration (nM, R=3 um): 18.69403433932219
Max mean concentration (nM, 10000 molecules per nucleus): 146.82388929884587
Max

In [5]:
bcd_htfs, bcd_htf_05 = get_htfs(10000, 600, prms.beta, prms.N_sites_tot)
print('htf values:\n', bcd_htfs, '\n')
print('htf_05:', bcd_htf_05)

Relative concentrations:
 [8.33333333e-04 2.50000000e-03 4.16666667e-03 5.83333333e-03
 7.50000000e-03 9.16666667e-03 1.08333333e-02 1.25000000e-02
 1.41666667e-02 1.58333333e-02 1.75000000e-02 1.91666667e-02
 2.08333333e-02 2.25000000e-02 2.41666667e-02 2.58333333e-02
 2.75000000e-02 2.91666667e-02 3.08333333e-02 3.25000000e-02
 3.41666667e-02 3.58333333e-02 3.75000000e-02 3.91666667e-02
 4.08333333e-02 4.25000000e-02 4.41666667e-02 4.58333333e-02
 4.75000000e-02 4.91666667e-02 5.08333333e-02 5.25000000e-02
 5.41666667e-02 5.58333333e-02 5.75000000e-02 5.91666667e-02
 6.08333333e-02 6.25000000e-02 6.41666667e-02 6.58333333e-02
 6.75000000e-02 6.91666667e-02 7.08333333e-02 7.25000000e-02
 7.41666667e-02 7.58333333e-02 7.75000000e-02 7.91666667e-02
 8.08333333e-02 8.25000000e-02 8.41666667e-02 8.58333333e-02
 8.75000000e-02 8.91666667e-02 9.08333333e-02 9.25000000e-02
 9.41666667e-02 9.58333333e-02 9.75000000e-02 9.91666667e-02
 1.00833333e-01 1.02500000e-01 1.04166667e-01 1.05833333e-0

## **Averaging time of 10 minutes**

In [38]:
## Grid
N             = 6
padding       = 0
periodic      = False
do_htf_update = False

L             = N+2*padding
sites         = np.arange(N)+padding
Grid_prms     = GridParameters(dim=1, L=L, sites=sites, IC_mode='probability', periodic=periodic, do_htf_update=do_htf_update)

## Gillespie
T             = prms.runtime/Grid_prms.time_conversion
Sim_prms      = SimulationParameters(runs=100, T=T)

## Variables
htf_list, htf_05 = get_htfs(10000, 60, Grid_prms.beta, Grid_prms.N_sites_tot)
J_list        = np.linspace(0, int(-htf_05), int(-htf_05)+1)
N_expr        = None
SC            = ShiftCompensation(Grid_prms, J_list, htf_05, N_expr=N_expr, T=None) # T=None --> Steady state
eb_list       = SC.find_eb()

if padding == 0:
    padding_str = 'NoPadding'
else:
    padding_str = f'{padding}Padding'
    
if periodic:
    periodic_str = 'Periodic'
else:
    periodic_str = 'NonPeriodic'
    
if do_htf_update:
    bath_str = 'FiniteBath'
else:
    bath_str = 'InfiniteBath'

if N_expr == None:
    expr_str = 'Occupancy'
else:
    expr_str = f'N{N_expr}Expression'

file_name     = f'StochasticModel/{N}Sites_{padding_str}_{periodic_str}_SteadyState_{bath_str}_{expr_str}_{date.today()}'
file          = os.path.join(os.path.realpath('__file__'), f'../../Data/{file_name}') 

f = open(file+'_shift_compensation', 'wb')
pickle.dump(SC, f)
f.close()

IC_prms_list  = get_steady_states(htf_list, J_list, eb_list, Grid_prms, file=file)
Varied_prms   = VariedParameters(htf_list, eb_list, J_list, IC_prms_list)

write_metadata(file, Grid_prms, Sim_prms, Varied_prms)
print('Done')

relative concentrations:
 [0.00833333 0.025      0.04166667 0.05833333 0.075      0.09166667
 0.10833333 0.125      0.14166667 0.15833333 0.175      0.19166667
 0.20833333 0.225      0.24166667 0.25833333 0.275      0.29166667
 0.30833333 0.325      0.34166667 0.35833333 0.375      0.39166667
 0.40833333 0.425      0.44166667 0.45833333 0.475      0.49166667
 0.50833333 0.525      0.54166667 0.55833333 0.575      0.59166667
 0.60833333 0.625      0.64166667 0.65833333 0.675      0.69166667
 0.70833333 0.725      0.74166667 0.75833333 0.775      0.79166667
 0.80833333 0.825      0.84166667 0.85833333 0.875      0.89166667
 0.90833333 0.925      0.94166667 0.95833333 0.975      0.99166667
 1.00833333 1.025      1.04166667 1.05833333 1.075     ] 

J=0.0, eb=16.24327397504225, htf=-16.86978290154109
J=0.0, eb=16.24327397504225, htf=-15.771170518558932
J=0.0, eb=16.24327397504225, htf=-15.260344800478883
J=0.0, eb=16.24327397504225, htf=-14.923872469543603
J=0.0, eb=16.24327397504225, htf=-

J=2.0, eb=13.545645283264296, htf=-13.373273831048756
J=2.0, eb=13.545645283264296, htf=-13.314433236711631
J=2.0, eb=13.545645283264296, htf=-13.25886329124262
J=2.0, eb=13.545645283264296, htf=-13.206219463442988
J=2.0, eb=13.545645283264296, htf=-13.156208948554108
J=2.0, eb=13.545645283264296, htf=-13.108580805250627
J=2.0, eb=13.545645283264296, htf=-13.063118336859633
J=2.0, eb=13.545645283264296, htf=-13.01963313060565
J=2.0, eb=13.545645283264296, htf=-12.977960339890828
J=2.0, eb=13.545645283264296, htf=-12.937954910962866
J=2.0, eb=13.545645283264296, htf=-12.899488535820797
J=2.0, eb=13.545645283264296, htf=-12.862447169826169
J=2.0, eb=13.545645283264296, htf=-12.8267289929098
J=2.0, eb=13.545645283264296, htf=-12.792242722524332
J=2.0, eb=13.545645283264296, htf=-12.758906207942433
J=2.0, eb=13.545645283264296, htf=-12.726645251409895
J=2.0, eb=13.545645283264296, htf=-12.695392613591466
J=2.0, eb=13.545645283264296, htf=-12.665087169781803
J=2.0, eb=13.545645283264296, ht

J=4.0, eb=11.410146495285723, htf=-12.500331370819557
J=4.0, eb=11.410146495285723, htf=-12.475329974299745
J=4.0, eb=11.410146495285723, htf=-12.45093842686118
J=4.0, eb=11.410146495285723, htf=-12.427127683853048
J=4.0, eb=11.410146495285723, htf=-12.403870727374358
J=4.0, eb=11.410146495285723, htf=-12.38114238198237
J=4.0, eb=11.410146495285723, htf=-12.358919150883219
J=4.0, eb=11.410146495285723, htf=-12.337179069932363
J=4.0, eb=11.410146495285723, htf=-12.31590157717062
J=4.0, eb=11.410146495285723, htf=-12.29506739595331
J=4.0, eb=11.410146495285723, htf=-12.274658430007628
J=4.0, eb=11.410146495285723, htf=-12.254657668986473
J=4.0, eb=11.410146495285723, htf=-12.235049103283602
J=4.0, eb=11.410146495285723, htf=-12.215817647041211
J=4.0, eb=11.410146495285723, htf=-12.196949068422317
J=4.0, eb=11.410146495285723, htf=-12.17842992634056
J=4.0, eb=11.410146495285723, htf=-12.160247512942838
J=4.0, eb=11.410146495285723, htf=-12.142389801228294
J=4.0, eb=11.410146495285723, htf

J=7.0, eb=8.544969384209544, htf=-14.672557946948622
J=7.0, eb=8.544969384209544, htf=-14.471887157172386
J=7.0, eb=8.544969384209544, htf=-14.304832978195126
J=7.0, eb=8.544969384209544, htf=-14.161732040240349
J=7.0, eb=8.544969384209544, htf=-14.036568802972232
J=7.0, eb=8.544969384209544, htf=-13.925343073547888
J=7.0, eb=8.544969384209544, htf=-13.825259520676775
J=7.0, eb=8.544969384209544, htf=-13.734287648156911
J=7.0, eb=8.544969384209544, htf=-13.650905944903712
J=7.0, eb=8.544969384209544, htf=-13.573944809453428
J=7.0, eb=8.544969384209544, htf=-13.502485751157119
J=7.0, eb=8.544969384209544, htf=-13.435794282344272
J=7.0, eb=8.544969384209544, htf=-13.373273831048756
J=7.0, eb=8.544969384209544, htf=-13.314433236711631
J=7.0, eb=8.544969384209544, htf=-13.25886329124262
J=7.0, eb=8.544969384209544, htf=-13.206219463442988
J=7.0, eb=8.544969384209544, htf=-13.156208948554108
J=7.0, eb=8.544969384209544, htf=-13.108580805250627
J=7.0, eb=8.544969384209544, htf=-13.0631183368

J=9.0, eb=6.772326936249609, htf=-12.552291298379034
J=9.0, eb=6.772326936249609, htf=-12.525973895747283
J=9.0, eb=6.772326936249609, htf=-12.500331370819557
J=9.0, eb=6.772326936249609, htf=-12.475329974299745
J=9.0, eb=6.772326936249609, htf=-12.45093842686118
J=9.0, eb=6.772326936249609, htf=-12.427127683853048
J=9.0, eb=6.772326936249609, htf=-12.403870727374358
J=9.0, eb=6.772326936249609, htf=-12.38114238198237
J=9.0, eb=6.772326936249609, htf=-12.358919150883219
J=9.0, eb=6.772326936249609, htf=-12.337179069932363
J=9.0, eb=6.772326936249609, htf=-12.31590157717062
J=9.0, eb=6.772326936249609, htf=-12.29506739595331
J=9.0, eb=6.772326936249609, htf=-12.274658430007628
J=9.0, eb=6.772326936249609, htf=-12.254657668986473
J=9.0, eb=6.772326936249609, htf=-12.235049103283602
J=9.0, eb=6.772326936249609, htf=-12.215817647041211
J=9.0, eb=6.772326936249609, htf=-12.196949068422317
J=9.0, eb=6.772326936249609, htf=-12.17842992634056
J=9.0, eb=6.772326936249609, htf=-12.16024751294283

J=12.0, eb=3.5700701650468223, htf=-16.86978290154109
J=12.0, eb=3.5700701650468223, htf=-15.771170518558932
J=12.0, eb=3.5700701650468223, htf=-15.260344800478883
J=12.0, eb=3.5700701650468223, htf=-14.923872469543603
J=12.0, eb=3.5700701650468223, htf=-14.672557946948622
J=12.0, eb=3.5700701650468223, htf=-14.471887157172386
J=12.0, eb=3.5700701650468223, htf=-14.304832978195126
J=12.0, eb=3.5700701650468223, htf=-14.161732040240349
J=12.0, eb=3.5700701650468223, htf=-14.036568802972232
J=12.0, eb=3.5700701650468223, htf=-13.925343073547888
J=12.0, eb=3.5700701650468223, htf=-13.825259520676775
J=12.0, eb=3.5700701650468223, htf=-13.734287648156911
J=12.0, eb=3.5700701650468223, htf=-13.650905944903712
J=12.0, eb=3.5700701650468223, htf=-13.573944809453428
J=12.0, eb=3.5700701650468223, htf=-13.502485751157119
J=12.0, eb=3.5700701650468223, htf=-13.435794282344272
J=12.0, eb=3.5700701650468223, htf=-13.373273831048756
J=12.0, eb=3.5700701650468223, htf=-13.314433236711631
J=12.0, eb=

J=14.0, eb=0.2389299943838612, htf=-13.108580805250627
J=14.0, eb=0.2389299943838612, htf=-13.063118336859633
J=14.0, eb=0.2389299943838612, htf=-13.01963313060565
J=14.0, eb=0.2389299943838612, htf=-12.977960339890828
J=14.0, eb=0.2389299943838612, htf=-12.937954910962866
J=14.0, eb=0.2389299943838612, htf=-12.899488535820797
J=14.0, eb=0.2389299943838612, htf=-12.862447169826169
J=14.0, eb=0.2389299943838612, htf=-12.8267289929098
J=14.0, eb=0.2389299943838612, htf=-12.792242722524332
J=14.0, eb=0.2389299943838612, htf=-12.758906207942433
J=14.0, eb=0.2389299943838612, htf=-12.726645251409895
J=14.0, eb=0.2389299943838612, htf=-12.695392613591466
J=14.0, eb=0.2389299943838612, htf=-12.665087169781803
J=14.0, eb=0.2389299943838612, htf=-12.635673190261167
J=14.0, eb=0.2389299943838612, htf=-12.60709972350276
J=14.0, eb=0.2389299943838612, htf=-12.579320065081323
J=14.0, eb=0.2389299943838612, htf=-12.552291298379034
J=14.0, eb=0.2389299943838612, htf=-12.525973895747283
J=14.0, eb=0.2

### Higher concentration sampling

In [3]:
## Grid
N             = 6
padding       = 0
periodic      = False
do_htf_update = False

L             = N+2*padding
sites         = np.arange(N)+padding
Grid_prms     = GridParameters(dim=1, L=L, sites=sites, IC_mode='probability', periodic=periodic, do_htf_update=do_htf_update)

## Gillespie
T             = prms.runtime/Grid_prms.time_conversion
Sim_prms      = SimulationParameters(runs=100, T=T)

## Variables
htf_list, htf_05 = get_htfs(10000, 600, Grid_prms.beta, Grid_prms.N_sites_tot)
J_list        = np.linspace(0, int(-htf_05), int(-htf_05)+1)
N_expr        = None
SC            = ShiftCompensation(Grid_prms, J_list, htf_05, N_expr=N_expr, T=None) # T=None --> Steady state
eb_list       = SC.find_eb()

if padding == 0:
    padding_str = 'NoPadding'
else:
    padding_str = f'{padding}Padding'
    
if periodic:
    periodic_str = 'Periodic'
else:
    periodic_str = 'NonPeriodic'
    
if do_htf_update:
    bath_str = 'FiniteBath'
else:
    bath_str = 'InfiniteBath'

if N_expr == None:
    expr_str = 'Occupancy'
else:
    expr_str = f'N{N_expr}Expression'

file_name     = f'{N}Sites_{padding_str}_{periodic_str}_SteadyState_{bath_str}_{expr_str}_{date.today()}'
file          = os.path.join(os.path.realpath('__file__'), f'../../Data/{file_name}') 

f = open(file+'_shift_compensation', 'wb')
pickle.dump(SC, f)
f.close()

IC_prms_list  = get_steady_states(htf_list, J_list, eb_list, Grid_prms, file=file)
Varied_prms   = VariedParameters(htf_list, eb_list, J_list, IC_prms_list)

write_metadata(file, Grid_prms, Sim_prms, Varied_prms)
print('Done')

Relative concentrations:
 [8.33333333e-04 2.50000000e-03 4.16666667e-03 5.83333333e-03
 7.50000000e-03 9.16666667e-03 1.08333333e-02 1.25000000e-02
 1.41666667e-02 1.58333333e-02 1.75000000e-02 1.91666667e-02
 2.08333333e-02 2.25000000e-02 2.41666667e-02 2.58333333e-02
 2.75000000e-02 2.91666667e-02 3.08333333e-02 3.25000000e-02
 3.41666667e-02 3.58333333e-02 3.75000000e-02 3.91666667e-02
 4.08333333e-02 4.25000000e-02 4.41666667e-02 4.58333333e-02
 4.75000000e-02 4.91666667e-02 5.08333333e-02 5.25000000e-02
 5.41666667e-02 5.58333333e-02 5.75000000e-02 5.91666667e-02
 6.08333333e-02 6.25000000e-02 6.41666667e-02 6.58333333e-02
 6.75000000e-02 6.91666667e-02 7.08333333e-02 7.25000000e-02
 7.41666667e-02 7.58333333e-02 7.75000000e-02 7.91666667e-02
 8.08333333e-02 8.25000000e-02 8.41666667e-02 8.58333333e-02
 8.75000000e-02 8.91666667e-02 9.08333333e-02 9.25000000e-02
 9.41666667e-02 9.58333333e-02 9.75000000e-02 9.91666667e-02
 1.00833333e-01 1.02500000e-01 1.04166667e-01 1.05833333e-0

J=0.0, eb=14.143319868324545, htf=-19.172368036976454
J=0.0, eb=14.143319868324545, htf=-18.073755738876944
J=0.0, eb=14.143319868324545, htf=-17.56293010567955
J=0.0, eb=14.143319868324545, htf=-17.22645785962693
J=0.0, eb=14.143319868324545, htf=-16.97514342191462
J=0.0, eb=14.143319868324545, htf=-16.774472717021062
J=0.0, eb=14.143319868324545, htf=-16.60741862292649
J=0.0, eb=14.143319868324545, htf=-16.464317769854414
J=0.0, eb=14.143319868324545, htf=-16.339154617469003
J=0.0, eb=14.143319868324545, htf=-16.227928972927376
J=0.0, eb=14.143319868324545, htf=-16.12784550493899
J=0.0, eb=14.143319868324545, htf=-16.036873717301855
J=0.0, eb=14.143319868324545, htf=-15.9534920989314
J=0.0, eb=14.143319868324545, htf=-15.876531048363866
J=0.0, eb=14.143319868324545, htf=-15.805072074950317
J=0.0, eb=14.143319868324545, htf=-15.73838069102024
J=0.0, eb=14.143319868324545, htf=-15.6758603246075
J=0.0, eb=14.143319868324545, htf=-15.61701981515316
J=0.0, eb=14.143319868324545, htf=-15.5

J=0.0, eb=14.143319868324545, htf=-13.413464772936221
J=0.0, eb=14.143319868324545, htf=-13.407175434597239
J=0.0, eb=14.143319868324545, htf=-13.400925404820649
J=0.0, eb=14.143319868324545, htf=-13.39471419529659
J=0.0, eb=14.143319868324545, htf=-13.38854132675809
J=0.0, eb=14.143319868324545, htf=-13.382406328759156
J=0.0, eb=14.143319868324545, htf=-13.376308739459619
J=0.0, eb=14.143319868324545, htf=-13.37024810541651
J=0.0, eb=14.143319868324545, htf=-13.36422398138171
J=0.0, eb=14.143319868324545, htf=-13.358235930105668
J=0.0, eb=14.143319868324545, htf=-13.352283522146953
J=0.0, eb=14.143319868324545, htf=-13.346366335687446
J=0.0, eb=14.143319868324545, htf=-13.34048395635296
J=0.0, eb=14.143319868324545, htf=-13.334635977039119
J=0.0, eb=14.143319868324545, htf=-13.32882199774228
J=0.0, eb=14.143319868324545, htf=-13.32304162539536
J=0.0, eb=14.143319868324545, htf=-13.317294473708372
J=0.0, eb=14.143319868324545, htf=-13.311580163013515
J=0.0, eb=14.143319868324545, htf=-

J=0.0, eb=14.143319868324545, htf=-12.72189463450415
J=0.0, eb=14.143319868324545, htf=-12.718740048324202
J=0.0, eb=14.143319868324545, htf=-12.715595382213298
J=0.0, eb=14.143319868324545, htf=-12.712460573976495
J=0.0, eb=14.143319868324545, htf=-12.709335562001927
J=0.0, eb=14.143319868324545, htf=-12.706220285253544
J=0.0, eb=14.143319868324545, htf=-12.703114683263959
J=0.0, eb=14.143319868324545, htf=-12.700018696127398
J=0.0, eb=14.143319868324545, htf=-12.696932264492775
J=0.0, eb=14.143319868324545, htf=-12.693855329556863
J=0.0, eb=14.143319868324545, htf=-12.690787833057568
J=0.0, eb=14.143319868324545, htf=-12.687729717267315
J=0.0, eb=14.143319868324545, htf=-12.684680924986523
J=0.0, eb=14.143319868324545, htf=-12.681641399537192
J=0.0, eb=14.143319868324545, htf=-12.67861108475658
J=0.0, eb=14.143319868324545, htf=-12.67558992499097
J=0.0, eb=14.143319868324545, htf=-12.672577865089545
J=0.0, eb=14.143319868324545, htf=-12.669574850398341
J=0.0, eb=14.143319868324545, h

J=0.0, eb=14.143319868324545, htf=-12.323297312141811
J=0.0, eb=14.143319868324545, htf=-12.32117865785008
J=0.0, eb=14.143319868324545, htf=-12.319064482726297
J=0.0, eb=14.143319868324545, htf=-12.316954767871001
J=0.0, eb=14.143319868324545, htf=-12.314849494504093
J=0.0, eb=14.143319868324545, htf=-12.312748643963834
J=0.0, eb=14.143319868324545, htf=-12.31065219770586
J=0.0, eb=14.143319868324545, htf=-12.30856013730219
J=0.0, eb=14.143319868324545, htf=-12.306472444440258
J=0.0, eb=14.143319868324545, htf=-12.304389100921957
J=0.0, eb=14.143319868324545, htf=-12.30231008866268
J=0.0, eb=14.143319868324545, htf=-12.300235389690371
J=0.0, eb=14.143319868324545, htf=-12.298164986144615
J=0.0, eb=14.143319868324545, htf=-12.296098860275697
J=0.0, eb=14.143319868324545, htf=-12.29403699444369
J=0.0, eb=14.143319868324545, htf=-12.291979371117563
J=0.0, eb=14.143319868324545, htf=-12.289925972874274
J=0.0, eb=14.143319868324545, htf=-12.287876782397891
J=0.0, eb=14.143319868324545, htf

...

J=13.0, eb=3.172406037359173, htf=-12.021660571561483
J=13.0, eb=3.172406037359173, htf=-12.02009316369001
J=13.0, eb=3.172406037359173, htf=-12.018528208712246
J=13.0, eb=3.172406037359173, htf=-12.016965698962894
J=13.0, eb=3.172406037359173, htf=-12.01540562681253
J=13.0, eb=3.172406037359173, htf=-12.013847984667386
J=13.0, eb=3.172406037359173, htf=-12.012292764969118
J=13.0, eb=3.172406037359173, htf=-12.010739960194597
J=13.0, eb=3.172406037359173, htf=-12.00918956285568
J=13.0, eb=3.172406037359173, htf=-12.007641565499004
J=14.0, eb=2.143424269595327, htf=-19.172368036976454
J=14.0, eb=2.143424269595327, htf=-18.073755738876944
J=14.0, eb=2.143424269595327, htf=-17.56293010567955
J=14.0, eb=2.143424269595327, htf=-17.22645785962693
J=14.0, eb=2.143424269595327, htf=-16.97514342191462
J=14.0, eb=2.143424269595327, htf=-16.774472717021062
J=14.0, eb=2.143424269595327, htf=-16.60741862292649
J=14.0, eb=2.143424269595327, htf=-16.464317769854414
J=14.0, eb=2.143424269595327, htf=-

J=14.0, eb=2.143424269595327, htf=-13.452054826794596
J=14.0, eb=2.143424269595327, htf=-13.445518846383393
J=14.0, eb=2.143424269595327, htf=-13.439025307641428
J=14.0, eb=2.143424269595327, htf=-13.43257366292852
J=14.0, eb=2.143424269595327, htf=-13.426163375136184
J=14.0, eb=2.143424269595327, htf=-13.419793917419286
J=14.0, eb=2.143424269595327, htf=-13.413464772936221
J=14.0, eb=2.143424269595327, htf=-13.407175434597239
J=14.0, eb=2.143424269595327, htf=-13.400925404820649
J=14.0, eb=2.143424269595327, htf=-13.39471419529659
J=14.0, eb=2.143424269595327, htf=-13.38854132675809
J=14.0, eb=2.143424269595327, htf=-13.382406328759156
J=14.0, eb=2.143424269595327, htf=-13.376308739459619
J=14.0, eb=2.143424269595327, htf=-13.37024810541651
J=14.0, eb=2.143424269595327, htf=-13.36422398138171
J=14.0, eb=2.143424269595327, htf=-13.358235930105668
J=14.0, eb=2.143424269595327, htf=-13.352283522146953
J=14.0, eb=2.143424269595327, htf=-13.346366335687446
J=14.0, eb=2.143424269595327, htf

J=14.0, eb=2.143424269595327, htf=-12.734613444637652
J=14.0, eb=2.143424269595327, htf=-12.731418544309701
J=14.0, eb=2.143424269595327, htf=-12.72823381881075
J=14.0, eb=2.143424269595327, htf=-12.725059203538544
J=14.0, eb=2.143424269595327, htf=-12.72189463450415
J=14.0, eb=2.143424269595327, htf=-12.718740048324202
J=14.0, eb=2.143424269595327, htf=-12.715595382213298
J=14.0, eb=2.143424269595327, htf=-12.712460573976495
J=14.0, eb=2.143424269595327, htf=-12.709335562001927
J=14.0, eb=2.143424269595327, htf=-12.706220285253544
J=14.0, eb=2.143424269595327, htf=-12.703114683263959
J=14.0, eb=2.143424269595327, htf=-12.700018696127398
J=14.0, eb=2.143424269595327, htf=-12.696932264492775
J=14.0, eb=2.143424269595327, htf=-12.693855329556863
J=14.0, eb=2.143424269595327, htf=-12.690787833057568
J=14.0, eb=2.143424269595327, htf=-12.687729717267315
J=14.0, eb=2.143424269595327, htf=-12.684680924986523
J=14.0, eb=2.143424269595327, htf=-12.681641399537192
J=14.0, eb=2.143424269595327, 

J=14.0, eb=2.143424269595327, htf=-12.333958441085153
J=14.0, eb=2.143424269595327, htf=-12.331817103212364
J=14.0, eb=2.143424269595327, htf=-12.329680340831183
J=14.0, eb=2.143424269595327, htf=-12.327548134429897
J=14.0, eb=2.143424269595327, htf=-12.325420464621335
J=14.0, eb=2.143424269595327, htf=-12.323297312141811
J=14.0, eb=2.143424269595327, htf=-12.32117865785008
J=14.0, eb=2.143424269595327, htf=-12.319064482726297
J=14.0, eb=2.143424269595327, htf=-12.316954767871001
J=14.0, eb=2.143424269595327, htf=-12.314849494504093
J=14.0, eb=2.143424269595327, htf=-12.312748643963834
J=14.0, eb=2.143424269595327, htf=-12.31065219770586
J=14.0, eb=2.143424269595327, htf=-12.30856013730219
J=14.0, eb=2.143424269595327, htf=-12.306472444440258
J=14.0, eb=2.143424269595327, htf=-12.304389100921957
J=14.0, eb=2.143424269595327, htf=-12.30231008866268
J=14.0, eb=2.143424269595327, htf=-12.300235389690371
J=14.0, eb=2.143424269595327, htf=-12.298164986144615
J=14.0, eb=2.143424269595327, ht

J=14.0, eb=2.143424269595327, htf=-12.048689404284223
J=14.0, eb=2.143424269595327, htf=-12.04707908854666
J=14.0, eb=2.143424269595327, htf=-12.045471361727104
J=14.0, eb=2.143424269595327, htf=-12.043866215514416
J=14.0, eb=2.143424269595327, htf=-12.042263641637415
J=14.0, eb=2.143424269595327, htf=-12.040663631864621
J=14.0, eb=2.143424269595327, htf=-12.039066178004004
J=14.0, eb=2.143424269595327, htf=-12.03747127190273
J=14.0, eb=2.143424269595327, htf=-12.035878905446907
J=14.0, eb=2.143424269595327, htf=-12.034289070561346
J=14.0, eb=2.143424269595327, htf=-12.032701759209314
J=14.0, eb=2.143424269595327, htf=-12.031116963392282
J=14.0, eb=2.143424269595327, htf=-12.029534675149693
J=14.0, eb=2.143424269595327, htf=-12.027954886558714
J=14.0, eb=2.143424269595327, htf=-12.026377589734006
J=14.0, eb=2.143424269595327, htf=-12.024802776827478
J=14.0, eb=2.143424269595327, htf=-12.023230440028065
J=14.0, eb=2.143424269595327, htf=-12.021660571561483
J=14.0, eb=2.143424269595327, 

## Time sweep

In [19]:
## Grid
N             = 6
padding       = 0
periodic      = False
do_htf_update = False

L             = N+2*padding
sites         = np.arange(N)+padding
Grid_prms     = GridParameters(dim=1, L=L, sites=sites, IC_mode='probability', periodic=periodic, do_htf_update=do_htf_update)

## Gillespie
T             = prms.runtime/Grid_prms.time_conversion

## Variables
_, htf_05     = get_htfs(10000, 60, Grid_prms.beta, Grid_prms.N_sites_tot)
htf_list      = np.linspace(-16, -12, 9)
J_list        = np.linspace(0, int(-htf_05), 5)
N_expr        = None
SC            = ShiftCompensation(Grid_prms, J_list, htf_05, N_expr=N_expr, T=None) # T=None --> Steady state
eb_list       = SC.find_eb()

if padding == 0:
    padding_str = 'NoPadding'
else:
    padding_str = f'{padding}Padding'
    
if periodic:
    periodic_str = 'Periodic'
else:
    periodic_str = 'NonPeriodic'
    
if do_htf_update:
    bath_str = 'FiniteBath'
else:
    bath_str = 'InfiniteBath'

if N_expr == None:
    expr_str = 'Occupancy'
else:
    expr_str = f'N{N_expr}Expression'

# Metadata files for time sweep were originally created in two parts, 
# which lead to numbering of files not bing in the same order as the averaging times.
# This numbering is used consistently throughout the scripts.
for i, t in enumerate(np.concatenante((np.logspace(-3,2,101), np.logspace(-6,-3,61)[:-1]))*T):
    Sim_prms = SimulationParameters(runs=100, T=t)
    
    file_name     = f'TimeSweep/TimeSweep_{N}Sites_{padding_str}_{periodic_str}_SteadyState_{bath_str}_{expr_str}_{date.today()}_{i}'
    file          = os.path.join(os.path.realpath('__file__'), f'../../Data/{file_name}') 

    IC_prms_list  = get_steady_states(htf_list, J_list, eb_list, Grid_prms, file=file)
    Varied_prms   = VariedParameters(htf_list, eb_list, J_list, IC_prms_list)
    
    f = open(file+'_shift_compensation', 'wb')
    pickle.dump(SC, f)
    f.close()

    write_metadata(file, Grid_prms, Sim_prms, Varied_prms)
    
print('Done')

relative concentrations:
 [0.00833333 0.025      0.04166667 0.05833333 0.075      0.09166667
 0.10833333 0.125      0.14166667 0.15833333 0.175      0.19166667
 0.20833333 0.225      0.24166667 0.25833333 0.275      0.29166667
 0.30833333 0.325      0.34166667 0.35833333 0.375      0.39166667
 0.40833333 0.425      0.44166667 0.45833333 0.475      0.49166667
 0.50833333 0.525      0.54166667 0.55833333 0.575      0.59166667
 0.60833333 0.625      0.64166667 0.65833333 0.675      0.69166667
 0.70833333 0.725      0.74166667 0.75833333 0.775      0.79166667
 0.80833333 0.825      0.84166667 0.85833333 0.875      0.89166667
 0.90833333 0.925      0.94166667 0.95833333 0.975      0.99166667
 1.00833333 1.025      1.04166667 1.05833333 1.075     ] 

J=0.0, eb=14.143319868324545, htf=-16.0
J=0.0, eb=14.143319868324545, htf=-15.5
J=0.0, eb=14.143319868324545, htf=-15.0
J=0.0, eb=14.143319868324545, htf=-14.5
J=0.0, eb=14.143319868324545, htf=-14.0
J=0.0, eb=14.143319868324545, htf=-13.5
J=0.0

J=0.0, eb=14.143319868324545, htf=-14.0
J=0.0, eb=14.143319868324545, htf=-13.5
J=0.0, eb=14.143319868324545, htf=-13.0
J=0.0, eb=14.143319868324545, htf=-12.5
J=0.0, eb=14.143319868324545, htf=-12.0
J=3.5, eb=11.140108819624526, htf=-16.0
J=3.5, eb=11.140108819624526, htf=-15.5
J=3.5, eb=11.140108819624526, htf=-15.0
J=3.5, eb=11.140108819624526, htf=-14.5
J=3.5, eb=11.140108819624526, htf=-14.0
J=3.5, eb=11.140108819624526, htf=-13.5
J=3.5, eb=11.140108819624526, htf=-13.0
J=3.5, eb=11.140108819624526, htf=-12.5
J=3.5, eb=11.140108819624526, htf=-12.0
J=7.0, eb=8.23962329975328, htf=-16.0
J=7.0, eb=8.23962329975328, htf=-15.5
J=7.0, eb=8.23962329975328, htf=-15.0
J=7.0, eb=8.23962329975328, htf=-14.5
J=7.0, eb=8.23962329975328, htf=-14.0
J=7.0, eb=8.23962329975328, htf=-13.5
J=7.0, eb=8.23962329975328, htf=-13.0
J=7.0, eb=8.23962329975328, htf=-12.5
J=7.0, eb=8.23962329975328, htf=-12.0
J=10.5, eb=5.352949615690948, htf=-16.0
J=10.5, eb=5.352949615690948, htf=-15.5
J=10.5, eb=5.35294

J=7.0, eb=8.23962329975328, htf=-13.0
J=7.0, eb=8.23962329975328, htf=-12.5
J=7.0, eb=8.23962329975328, htf=-12.0
J=10.5, eb=5.352949615690948, htf=-16.0
J=10.5, eb=5.352949615690948, htf=-15.5
J=10.5, eb=5.352949615690948, htf=-15.0
J=10.5, eb=5.352949615690948, htf=-14.5
J=10.5, eb=5.352949615690948, htf=-14.0
J=10.5, eb=5.352949615690948, htf=-13.5
J=10.5, eb=5.352949615690948, htf=-13.0
J=10.5, eb=5.352949615690948, htf=-12.5
J=10.5, eb=5.352949615690948, htf=-12.0
J=14.0, eb=2.4541905752958177, htf=-16.0
J=14.0, eb=2.4541905752958177, htf=-15.5
J=14.0, eb=2.4541905752958177, htf=-15.0
J=14.0, eb=2.4541905752958177, htf=-14.5
J=14.0, eb=2.4541905752958177, htf=-14.0
J=14.0, eb=2.4541905752958177, htf=-13.5
J=14.0, eb=2.4541905752958177, htf=-13.0
J=14.0, eb=2.4541905752958177, htf=-12.5
J=14.0, eb=2.4541905752958177, htf=-12.0
J=0.0, eb=14.143319868324545, htf=-16.0
J=0.0, eb=14.143319868324545, htf=-15.5
J=0.0, eb=14.143319868324545, htf=-15.0
J=0.0, eb=14.143319868324545, htf=-14

J=3.5, eb=11.140108819624526, htf=-14.0
J=3.5, eb=11.140108819624526, htf=-13.5
J=3.5, eb=11.140108819624526, htf=-13.0
J=3.5, eb=11.140108819624526, htf=-12.5
J=3.5, eb=11.140108819624526, htf=-12.0
J=7.0, eb=8.23962329975328, htf=-16.0
J=7.0, eb=8.23962329975328, htf=-15.5
J=7.0, eb=8.23962329975328, htf=-15.0
J=7.0, eb=8.23962329975328, htf=-14.5
J=7.0, eb=8.23962329975328, htf=-14.0
J=7.0, eb=8.23962329975328, htf=-13.5
J=7.0, eb=8.23962329975328, htf=-13.0
J=7.0, eb=8.23962329975328, htf=-12.5
J=7.0, eb=8.23962329975328, htf=-12.0
J=10.5, eb=5.352949615690948, htf=-16.0
J=10.5, eb=5.352949615690948, htf=-15.5
J=10.5, eb=5.352949615690948, htf=-15.0
J=10.5, eb=5.352949615690948, htf=-14.5
J=10.5, eb=5.352949615690948, htf=-14.0
J=10.5, eb=5.352949615690948, htf=-13.5
J=10.5, eb=5.352949615690948, htf=-13.0
J=10.5, eb=5.352949615690948, htf=-12.5
J=10.5, eb=5.352949615690948, htf=-12.0
J=14.0, eb=2.4541905752958177, htf=-16.0
J=14.0, eb=2.4541905752958177, htf=-15.5
J=14.0, eb=2.454

J=7.0, eb=8.23962329975328, htf=-15.5
J=7.0, eb=8.23962329975328, htf=-15.0
J=7.0, eb=8.23962329975328, htf=-14.5
J=7.0, eb=8.23962329975328, htf=-14.0
J=7.0, eb=8.23962329975328, htf=-13.5
J=7.0, eb=8.23962329975328, htf=-13.0
J=7.0, eb=8.23962329975328, htf=-12.5
J=7.0, eb=8.23962329975328, htf=-12.0
J=10.5, eb=5.352949615690948, htf=-16.0
J=10.5, eb=5.352949615690948, htf=-15.5
J=10.5, eb=5.352949615690948, htf=-15.0
J=10.5, eb=5.352949615690948, htf=-14.5
J=10.5, eb=5.352949615690948, htf=-14.0
J=10.5, eb=5.352949615690948, htf=-13.5
J=10.5, eb=5.352949615690948, htf=-13.0
J=10.5, eb=5.352949615690948, htf=-12.5
J=10.5, eb=5.352949615690948, htf=-12.0
J=14.0, eb=2.4541905752958177, htf=-16.0
J=14.0, eb=2.4541905752958177, htf=-15.5
J=14.0, eb=2.4541905752958177, htf=-15.0
J=14.0, eb=2.4541905752958177, htf=-14.5
J=14.0, eb=2.4541905752958177, htf=-14.0
J=14.0, eb=2.4541905752958177, htf=-13.5
J=14.0, eb=2.4541905752958177, htf=-13.0
J=14.0, eb=2.4541905752958177, htf=-12.5
J=14.0, 

...

J=3.5, eb=11.140108819624526, htf=-12.0
J=7.0, eb=8.23962329975328, htf=-16.0
J=7.0, eb=8.23962329975328, htf=-15.5
J=7.0, eb=8.23962329975328, htf=-15.0
J=7.0, eb=8.23962329975328, htf=-14.5
J=7.0, eb=8.23962329975328, htf=-14.0
J=7.0, eb=8.23962329975328, htf=-13.5
J=7.0, eb=8.23962329975328, htf=-13.0
J=7.0, eb=8.23962329975328, htf=-12.5
J=7.0, eb=8.23962329975328, htf=-12.0
J=10.5, eb=5.352949615690948, htf=-16.0
J=10.5, eb=5.352949615690948, htf=-15.5
J=10.5, eb=5.352949615690948, htf=-15.0
J=10.5, eb=5.352949615690948, htf=-14.5
J=10.5, eb=5.352949615690948, htf=-14.0
J=10.5, eb=5.352949615690948, htf=-13.5
J=10.5, eb=5.352949615690948, htf=-13.0
J=10.5, eb=5.352949615690948, htf=-12.5
J=10.5, eb=5.352949615690948, htf=-12.0
J=14.0, eb=2.4541905752958177, htf=-16.0
J=14.0, eb=2.4541905752958177, htf=-15.5
J=14.0, eb=2.4541905752958177, htf=-15.0
J=14.0, eb=2.4541905752958177, htf=-14.5
J=14.0, eb=2.4541905752958177, htf=-14.0
J=14.0, eb=2.4541905752958177, htf=-13.5
J=14.0, eb=2

J=7.0, eb=8.23962329975328, htf=-16.0
J=7.0, eb=8.23962329975328, htf=-15.5
J=7.0, eb=8.23962329975328, htf=-15.0
J=7.0, eb=8.23962329975328, htf=-14.5
J=7.0, eb=8.23962329975328, htf=-14.0
J=7.0, eb=8.23962329975328, htf=-13.5
J=7.0, eb=8.23962329975328, htf=-13.0
J=7.0, eb=8.23962329975328, htf=-12.5
J=7.0, eb=8.23962329975328, htf=-12.0
J=10.5, eb=5.352949615690948, htf=-16.0
J=10.5, eb=5.352949615690948, htf=-15.5
J=10.5, eb=5.352949615690948, htf=-15.0
J=10.5, eb=5.352949615690948, htf=-14.5
J=10.5, eb=5.352949615690948, htf=-14.0
J=10.5, eb=5.352949615690948, htf=-13.5
J=10.5, eb=5.352949615690948, htf=-13.0
J=10.5, eb=5.352949615690948, htf=-12.5
J=10.5, eb=5.352949615690948, htf=-12.0
J=14.0, eb=2.4541905752958177, htf=-16.0
J=14.0, eb=2.4541905752958177, htf=-15.5
J=14.0, eb=2.4541905752958177, htf=-15.0
J=14.0, eb=2.4541905752958177, htf=-14.5
J=14.0, eb=2.4541905752958177, htf=-14.0
J=14.0, eb=2.4541905752958177, htf=-13.5
J=14.0, eb=2.4541905752958177, htf=-13.0
J=14.0, eb=

J=3.5, eb=11.140108819624526, htf=-16.0
J=3.5, eb=11.140108819624526, htf=-15.5
J=3.5, eb=11.140108819624526, htf=-15.0
J=3.5, eb=11.140108819624526, htf=-14.5
J=3.5, eb=11.140108819624526, htf=-14.0
J=3.5, eb=11.140108819624526, htf=-13.5
J=3.5, eb=11.140108819624526, htf=-13.0
J=3.5, eb=11.140108819624526, htf=-12.5
J=3.5, eb=11.140108819624526, htf=-12.0
J=7.0, eb=8.23962329975328, htf=-16.0
J=7.0, eb=8.23962329975328, htf=-15.5
J=7.0, eb=8.23962329975328, htf=-15.0
J=7.0, eb=8.23962329975328, htf=-14.5
J=7.0, eb=8.23962329975328, htf=-14.0
J=7.0, eb=8.23962329975328, htf=-13.5
J=7.0, eb=8.23962329975328, htf=-13.0
J=7.0, eb=8.23962329975328, htf=-12.5
J=7.0, eb=8.23962329975328, htf=-12.0
J=10.5, eb=5.352949615690948, htf=-16.0
J=10.5, eb=5.352949615690948, htf=-15.5
J=10.5, eb=5.352949615690948, htf=-15.0
J=10.5, eb=5.352949615690948, htf=-14.5
J=10.5, eb=5.352949615690948, htf=-14.0
J=10.5, eb=5.352949615690948, htf=-13.5
J=10.5, eb=5.352949615690948, htf=-13.0
J=10.5, eb=5.35294

J=0.0, eb=14.143319868324545, htf=-14.0
J=0.0, eb=14.143319868324545, htf=-13.5
J=0.0, eb=14.143319868324545, htf=-13.0
J=0.0, eb=14.143319868324545, htf=-12.5
J=0.0, eb=14.143319868324545, htf=-12.0
J=3.5, eb=11.140108819624526, htf=-16.0
J=3.5, eb=11.140108819624526, htf=-15.5
J=3.5, eb=11.140108819624526, htf=-15.0
J=3.5, eb=11.140108819624526, htf=-14.5
J=3.5, eb=11.140108819624526, htf=-14.0
J=3.5, eb=11.140108819624526, htf=-13.5
J=3.5, eb=11.140108819624526, htf=-13.0
J=3.5, eb=11.140108819624526, htf=-12.5
J=3.5, eb=11.140108819624526, htf=-12.0
J=7.0, eb=8.23962329975328, htf=-16.0
J=7.0, eb=8.23962329975328, htf=-15.5
J=7.0, eb=8.23962329975328, htf=-15.0
J=7.0, eb=8.23962329975328, htf=-14.5
J=7.0, eb=8.23962329975328, htf=-14.0
J=7.0, eb=8.23962329975328, htf=-13.5
J=7.0, eb=8.23962329975328, htf=-13.0
J=7.0, eb=8.23962329975328, htf=-12.5
J=7.0, eb=8.23962329975328, htf=-12.0
J=10.5, eb=5.352949615690948, htf=-16.0
J=10.5, eb=5.352949615690948, htf=-15.5
J=10.5, eb=5.35294

J=3.5, eb=11.140108819624526, htf=-16.0
J=3.5, eb=11.140108819624526, htf=-15.5
J=3.5, eb=11.140108819624526, htf=-15.0
J=3.5, eb=11.140108819624526, htf=-14.5
J=3.5, eb=11.140108819624526, htf=-14.0
J=3.5, eb=11.140108819624526, htf=-13.5
J=3.5, eb=11.140108819624526, htf=-13.0
J=3.5, eb=11.140108819624526, htf=-12.5
J=3.5, eb=11.140108819624526, htf=-12.0
J=7.0, eb=8.23962329975328, htf=-16.0
J=7.0, eb=8.23962329975328, htf=-15.5
J=7.0, eb=8.23962329975328, htf=-15.0
J=7.0, eb=8.23962329975328, htf=-14.5
J=7.0, eb=8.23962329975328, htf=-14.0
J=7.0, eb=8.23962329975328, htf=-13.5
J=7.0, eb=8.23962329975328, htf=-13.0
J=7.0, eb=8.23962329975328, htf=-12.5
J=7.0, eb=8.23962329975328, htf=-12.0
J=10.5, eb=5.352949615690948, htf=-16.0
J=10.5, eb=5.352949615690948, htf=-15.5
J=10.5, eb=5.352949615690948, htf=-15.0
J=10.5, eb=5.352949615690948, htf=-14.5
J=10.5, eb=5.352949615690948, htf=-14.0
J=10.5, eb=5.352949615690948, htf=-13.5
J=10.5, eb=5.352949615690948, htf=-13.0
J=10.5, eb=5.35294